In [4]:
import yfinance as yf 
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, timedelta

In [9]:
key = 'zAhuvSGdkpJ500Vq7cOSYp6iONhWh8NY'

def get_polygon_data(ticker, multiplier=1, timespan="day", from_date=None, to_date=None, limit=100):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{multiplier}/{timespan}/{from_date}/{to_date}"
    params = {"adjusted": "true", "sort": "desc", "limit": limit, "apiKey": key}
    response = requests.get(url, params=params)
    data = response.json()
    if "results" in data:
        df = pd.DataFrame(data["results"])
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        df.set_index('t', inplace=True)
        df = df.sort_index()
        df.rename(columns={'c': 'close'}, inplace=True)
        return df[['close']]
    else:
        raise Exception(f"Error: {data.get('error', 'Unknown error')}")

def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def generate_dca_signal():
    today = datetime.today().date()
    from_date = (today - timedelta(days=30)).isoformat()
    to_date = today.isoformat()

    spy_df = get_polygon_data("SPY", from_date=from_date, to_date=to_date)
    spy_df['rsi'] = calculate_rsi(spy_df['close'])

    # Weekly return (last close vs 5 days ago)
    if len(spy_df) < 6:
        print("Not enough data.")
        return

    weekly_return = (spy_df['close'].iloc[-1] - spy_df['close'].iloc[-6]) / spy_df['close'].iloc[-6] * 100
    latest_rsi = spy_df['rsi'].iloc[-1]

    # VIX from Polygon (assume VIX = ^VIX)
    spy_df['range'] = spy_df['close'].pct_change().abs()
    avg_range = spy_df['range'].tail(10).mean() * 100  # in %

    print(f"\nWeekly SPY Change: {weekly_return:.2f}%")
    print(f"RSI: {latest_rsi:.2f}")
    print(f"VOL: {avg_range:.2f}")

    signal_count = 0
    if weekly_return <= -3:
        print("✔ SPY dropped >3% this week")
        signal_count += 1
    if latest_rsi < 30:
        print("✔ RSI < 30 (Oversold)")
        signal_count += 1
    if avg_range > 1.5:
        print("High Volatility, avg range >1.5%")
        signal_count += 1

    print("\n📌 Recommendation:")
    if signal_count == 0:
        print("❌ No buy signal")
    elif signal_count == 1:
        print("🟡 Partial Buy (1 signal hit)")
    elif signal_count == 2:
        print("🟠 Stronger Buy (2 signals hit)")
    else:
        print("🟢 Full Buy (All 3 signals hit)")

generate_dca_signal()


Weekly SPY Change: -0.86%
RSI: 69.90
VOL: 0.68

📌 Recommendation:
❌ No buy signal
